In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style="whitegrid")
sns.set_theme()

In [2]:
df = pd.read_csv('transcriptomics_fractions.csv')
df1 = pd.read_csv('samples_transcriptomics.csv', index_col='Unnamed: 0').set_index('Sample.ID')
df2 = pd.read_csv('Ecoli_gene_functional_annotations.csv', encoding = 'cp1252', index_col='Unnamed: 0')

In [3]:
df_renamed = df.rename(columns={'Unnamed: 0': 'gene'})
df_T = df_renamed.set_index('gene')
df2_sorted = df2.dropna(subset=['gene']).sort_values(by='gene', ascending=True).reset_index(drop=True)
df2_s = df2_sorted.set_index('gene')

In [4]:
df_join = pd.merge(df2_s, df_T, left_index=True, right_index=True)

In [5]:
Alim = df_join.loc[:,:'a4_1'].sort_values(by='top_cog_functional_cat', ascending=True)
Clim = df_join.drop(df_join.columns[list(range(7,14)) + list(range(24, 35))], axis=1).sort_values(by='top_cog_functional_cat', ascending=True)
Rlim = df_join.drop(df_join.columns[7:24], axis=1).sort_values(by='top_cog_functional_cat', ascending=True)

In [ ]:
Alim_func_category_description = list(dict.fromkeys(Alim['functional_category_desc'].tolist()))
Clim_func_category_description = list(dict.fromkeys(Clim['functional_category_desc'].tolist()))
Rlim_func_category_description = list(dict.fromkeys(Rlim['functional_category_desc'].tolist()))
func = list(dict.fromkeys(Alim['top_cog_functional_cat'].tolist()))

In [9]:
def ffunctionality(group, functionality, a ,b):
    'a, b =  0, 7 for Alim'
    'a, b =  7, 17 for Clim'
    'a, b =  17, 28 for Rlim'
    func = group['top_cog_functional_cat'].tolist()
    for raw in enumerate(func):
        group_func = group[group['top_cog_functional_cat']==str(functionality)]
        group_f= group_func.iloc[:,7:].T
        #group_f['Growth.rate']= list(df1['Growth.rate..1.h.'][a:b])
        group_ff[] = group_f.sum(axis=1)
    return group_ff#pd.merge(df1[:][:7], group_ff, left_index=True, right_index=True)

In [7]:
def Tcog_func(G_name, group, a, b):
    func = list(dict.fromkeys(Alim['top_cog_functional_cat'].tolist()))
    df_dict = {}
    for i in func:
        df_result = ffunctionality(group, i, a, b)
        df_dict[f"{G_name}_{i}"] = df_result
        #print(f"{G_name}_{i}")
    return df_dict

In [ ]:
Tcog_func_by_functionality = Tcog_func("Alim", Alim, 0, 7)
for key, df in Tcog_func_by_functionality.items():
    print(f"{key}:\n", df)

In [ ]:
Tcog_func_by_functionality.items()

In [11]:
AlimA = ffunctionality(Alim, 'C', 0 , 7)
AlimA

gene,xdhD,xdhC,xdhB,nuoL,xdhA,nuoM,ygcN,ygcO,azoR,nuoN,...,dcuD,dcuC,gpsA,ldhA,prpC,dctA,leuB,eutG,appC,yieF
a1,0.000004,0.000003,0.000003,0.000129,0.000003,0.000186,0.000005,0.000003,0.000330,0.000243,...,0.000003,0.000003,0.000258,0.000016,5.970000e-07,0.000041,0.000449,0.000001,0.000002,0.000062
a1_1,0.000004,0.000004,0.000003,0.000153,0.000003,0.000226,0.000006,0.000002,0.000351,0.000311,...,0.000003,0.000016,0.000251,0.000030,7.430000e-07,0.000019,0.000340,0.000001,0.000002,0.000102
a2,0.000006,0.000006,0.000005,0.000152,0.000005,0.000215,0.000006,0.000004,0.000555,0.000286,...,0.000003,0.000004,0.000267,0.000019,1.180000e-06,0.000079,0.000611,0.000002,0.000004,0.000123
a2_1,0.000005,0.000005,0.000004,0.000167,0.000005,0.000253,0.000011,0.000005,0.000400,0.000352,...,0.000004,0.000012,0.000321,0.000017,9.990000e-07,0.000064,0.000542,0.000001,0.000004,0.000126
a3,0.000006,0.000006,0.000005,0.000128,0.000007,0.000209,0.000010,0.000005,0.000381,0.000305,...,0.000004,0.000016,0.000252,0.000021,1.210000e-06,0.000211,0.000424,0.000002,0.000005,0.000163
a4,0.000007,0.000009,0.000005,0.000157,0.000008,0.000238,0.000007,0.000004,0.000531,0.000301,...,0.000004,0.000010,0.000246,0.000026,1.590000e-06,0.000178,0.000556,0.000002,0.000006,0.000153
a4_1,0.000006,0.000007,0.000005,0.000167,0.000008,0.000263,0.000008,0.000004,0.000247,0.000296,...,0.000003,0.000017,0.000270,0.000022,1.730000e-06,0.000202,0.000413,0.000002,0.000005,0.000162


In [ ]:
AlimC = ffunctionality(Alim, 'C', 0 , 7)

In [ ]:
AlimCE = ffunctionality(Alim, 'D', 0 , 7)

In [ ]:
plt.figure(figsize=(8, 5))

for col in AlimCE.columns[9:-1]: 
    sns.lineplot(x=AlimCE['Growth.rate'], y=AlimCE[col], label=col, marker='o')

plt.xlabel('Lambda')
plt.ylabel('Conc')
plt.title('Functionality CE')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
def sepffuncionality(group, functionality, a, b):
    func = group['cog_functional_cat'].tolist()
    for i in func:
        a = ffunctionality(group, functionality, a ,b)

In [ ]:
func

In [ ]:
func = list(dict.fromkeys(Alim['cog_functional_cat'].tolist()))
for i in func:
    Alim1 = str('Alim') + str(i) 
    A = ffunctionality(Alim, i, 0, 7)
    print(Alim1)
    print(A)

In [ ]:
func = list(dict.fromkeys(Alim['cog_functional_cat'].tolist()))

In [ ]:
cols_per_figure = 4 

for i in range(1, Alim.shape[1]-1, cols_per_figure):
    fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16, 3))
    axes = axes.flatten()  
    
    for j, col in enumerate(Alim.columns[i:i+cols_per_figure]):
        ax = axes[j]
        sns.lineplot(x=Alim['Growth.rate'], y=Alim[col], ax=ax, marker='o')
        ax.set_title(f'{col}')
        ax.set_xlabel('Lambda')
        ax.set_ylabel('Conc.')
    
    plt.tight_layout()
    plt.show()

In [ ]:
cols_per_figure = 4
RlimC = ffunctionality(Rlim_sor, str(for i in func: print(i) , 17, 28)

for i in range(0, RlimC.shape[1]-1, cols_per_figure):
    fig, axes = plt.subplots(nrows=1, ncols=Alim_fA.shape[1]-1, figsize=(16, 3))
    axes = axes.flatten()  
    
    for j, col in enumerate(Alim_fa.columns[i:i+cols_per_figure]):
        ax = axes[j]
        sns.lineplot(x=Alim_fa['Growth.rate'], y=Alim_fa[col], ax=ax, marker='o')
        ax.set_title(f'Alim - A- {col}')
        #ax.set_xlabel('Lambda')
        ax.set_ylabel('Conc.')
    
    plt.tight_layout()
    plt.show()

In [ ]:
df1_plus_AlimFa = df1[:][:7].merge(Alim_fa, how='inner', on='Sample.ID')
df1_plus_AlimFa

In [ ]:
Alim_Sumplement60 = df1_plus_AlimFa[df1_plus_AlimFa['Supplement']=='60 µM IPTG']
Alim_Sumplement40 = df1_plus_AlimFa[df1_plus_AlimFa['Supplement']=='40 µM IPTG']
Alim_Sumplement28 = df1_plus_AlimFa[df1_plus_AlimFa['Supplement']=='28 µM IPTG']
Alim_Sumplement20 = df1_plus_AlimFa[df1_plus_AlimFa['Supplement']=='20 µM IPTG']

In [ ]:
sns.scatterplot(x = Alim_fa['Growth.rate'], y = Alim_fa['rtcA']+Alim_fa['orn'], marker='o')
#sns.lineplot(x = Alim_fa['Growth.rate'], y = Alim_fa['orn'], marker='o')
#sns.lineplot(Alim_Sumplement28['Growth.rate'], df['orn'], marker='o')
#sns.lineplot(Alim_Sumplement20['Growth.rate'], df['orn'], marker='o')
plt.xlabel('Lambda')
plt.ylabel('Conc')
plt.title(f'Alim - A-')
plt.legend()
plt.show()

In [ ]:
cols_per_figure = 2

for i in range(0, Alim_fA.shape[1]-1, cols_per_figure):
    fig, axes = plt.subplots(nrows=1, ncols=Alim_fA.shape[1]-1, figsize=(16, 3))
    axes = axes.flatten()  
    
    for j, col in enumerate(Alim_fa.columns[i:i+cols_per_figure]):
        ax = axes[j]
        sns.lineplot(x=Alim_fa['Growth.rate'], y=Alim_fa[col], ax=ax, marker='o')
        ax.set_title(f'Alim - A- {col}')
        #ax.set_xlabel('Lambda')
        ax.set_ylabel('Conc.')
    
    plt.tight_layout()
    plt.show()

In [ ]:
 Alim_fA.shape[1]

In [ ]:
import pandas as pd

# Crear un DataFrame de ejemplo con 20 columnas
data = {f'Col{i}': range(1, 6) for i in range(1, 21)}
df = pd.DataFrame(data)

# Eliminar columnas del 5 al 8 (índices 4 a 7) y del 14 al 20 (índices 13 a 19)
df_filtrado = df.drop(df.columns[list(range(4, 8)) + list(range(13, 19))], axis=1)

print(df_filtrado)